In [ ]:
import os
import json
import requests
from functools import lru_cache

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
tqdm.pandas(desc="Processing rows")

MIN_DRUG_OCCURENCE_THRESHOLD = 10

In [ ]:
# Merge hand curated field names

fields = []
fields_by_file = {}  # for helper files

for filename in sorted(os.listdir('data/raw/ukb_export/')):
    if not filename.startswith('field') or not filename.endswith('txt'):
        continue

    if filename == 'field.txt':  # avoid field.txt if it was already created
        continue 
    
    with open(f'data/raw/ukb_export/{filename}') as f:
        field_ids = [int(line.strip()) for line in f.readlines()]
        fields += field_ids

        fields_by_file[filename] = field_ids

with open('data/raw/ukb_export/field.txt', 'wt') as f:
    for field_id in fields:
        f.write(f'{field_id}\n')

run data/raw/ukb_export/extract.sh

# Set column dtypes and data encoding for exported columns

In [ ]:
data_encoding_df = pd.read_csv('data/raw/helper_files/Codings.tsv', sep='\t')
field_info_df = pd.read_csv('data/raw/helper_files/Data_Dictionary_Showcase.tsv', sep='\t').set_index('FieldID')

exported_df = pd.read_csv('data/raw/ukb_export/data.csv').set_index('eid')

# rename 189 to 22189, legacy downloads...
exported_df.rename(columns={col: '22'+col for col in exported_df if col.split('-')[0] == '189'}, inplace=True)

# drop people with no birth information
exported_df = exported_df[~exported_df['52-0.0'].isna()]

In [ ]:
categorical_dtypes = {}
special_values = {}  # for helper files

for col in tqdm(exported_df.columns):
    field_id = int(col.split('-')[0])
    try:
        field_info = field_info_df.loc[field_id]
    except KeyError:
        print('ERROR', field_id)
        continue

    has_coding = not pd.isna(field_info['Coding'])

    if has_coding:
        coding = int(field_info['Coding'])

    if 'Categorical' in field_info['ValueType']:  # Switch 'Categorical single' columns to pandas categorical dtypes
        if coding not in categorical_dtypes:
            mapping = {int(row['Value']): row['Meaning'] for _, row in data_encoding_df[data_encoding_df['Coding'] == coding].iterrows()}
            categorical_dtypes[coding] = (pd.CategoricalDtype(mapping.values()), mapping)
        dtype, mapping = categorical_dtypes[coding]
        exported_df[col] = exported_df[col].map(mapping).astype(dtype)

    # save 'Integer' and 'Continuous' with special data encodings
    if field_info['ValueType'] == 'Integer': 
        if has_coding:
            special_values[field_id] = {int(row['Value']): row['Meaning'] for _, row in data_encoding_df[data_encoding_df['Coding'] == coding].iterrows()}
        exported_df[col] = exported_df[col].astype('Int64') # Switch 'Integer' columns to pandas Int64
            
    elif field_info['ValueType'] == 'Continuous':
        if has_coding:
            special_values[field_id] = {float(row['Value']): row['Meaning'] for _, row in data_encoding_df[data_encoding_df['Coding'] == coding].iterrows()}

## Calculate estimated date of events missing exact information

For various reasons the certain date of some events is not accessible, these field are estimated from available information. The official UKB field ids are used when imputing the missing dates, and the extra information is then dropped

Date of birth
https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=33

Reception Date
https://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=53

In [ ]:
birth_dates = pd.to_datetime(exported_df[['34-0.0', '52-0.0']].progress_apply(lambda row: f"{row['34-0.0']} {row['52-0.0']}", axis=1), format='%Y %B')
exported_df['33-0.0'] = birth_dates.dt.strftime('%Y-%m')
exported_df = exported_df.drop(['34-0.0', '52-0.0'], axis=1)

In [ ]:
# different UKB instances are from different assesments

# 0 Initial assessment visit (2006-2010) at which participants were recruited and consent given
# 1	rep1	First repeat assessment visit (2012-13)
# 2	img	Imaging visit (2014+)
# 3	irep1	First repeat imaging visit (2019+)

assesment_dates = [
    (pd.to_datetime('2006-01-01'), pd.to_datetime('2010-12-31')),
    (pd.to_datetime('2012-01-01'), pd.to_datetime('2013-12-31')),
    (pd.to_datetime('2014-01-01'), pd.to_datetime('NaT')),
    (pd.to_datetime('2019-01-01'), pd.to_datetime('NaT')),
]

assesment_cols = []

for c in exported_df:
    field_id, instance, array = map(int, c.replace('.', '-').split('-'))
    
    if field_id != 21003:
        continue

    assesment_cols.append(c)
    
    start_date, end_date = assesment_dates[instance]

    date_offset = exported_df[c].progress_apply(
        lambda x: pd.DateOffset(years=x) if pd.notna(x) else pd.NaT
    )
    
    min_dates = birth_dates + date_offset
    max_dates = min_dates + pd.DateOffset(years=1, months=1)
    
    min_dates[min_dates < start_date] = start_date
    max_dates[max_dates > end_date] = end_date

    average_dates = min_dates + (max_dates - min_dates) / 2

    exported_df[f'{53}-{instance}.{array}'] = average_dates.dt.strftime('%Y-%m-%d')

exported_df = exported_df.drop(assesment_cols, axis=1)

# Add events

In [ ]:
def handle_event_df(event_df):
    # https://biobank.ndph.ox.ac.uk/showcase/coding.cgi?id=819
    date_indicating_missing_information = [
        '1900-01-01',
        '1901-01-01',
        '1909-09-09',
        '2037-07-07',
    ]
    
    date_matching_birthdate = '1902-02-02'
    date_matching_birthyear = '1903-03-03'
    
    event_df = event_df[~event_df['date'].isna()]
    event_df['date'] = event_df['date'].progress_map(lambda date: '-'.join(date.split('/')[::-1]))
    event_df = event_df[~event_df['date'].isin(date_indicating_missing_information)]
    
    mask = event_df['date'] == date_matching_birthdate
    event_df.loc[mask, 'date'] = exported_df.loc[event_df.loc[mask].index, '33-0.0']
    
    mask = event_df['date'] == date_matching_birthyear
    event_df.loc[mask, 'date'] = exported_df.loc[event_df.loc[mask].index, '33-0.0'].apply(lambda date: date.split('-')[0])
    
    event_df = event_df.groupby('eid').agg({'event': list, 'date': list})
    
    def parse_and_sort_dates(row):
        # TODO this will handle "date_matching_birthyear" cases incorrectly and put them before any birth events
        dates = np.array(row['date'])
        events = np.array(row['event'])
    
        sorted_indices = np.argsort(dates)
        return {'event': events[sorted_indices], 'date': dates[sorted_indices]}
    
    event_df = event_df.progress_apply(parse_and_sort_dates, axis=1, result_type='expand')

    return event_df

## Convert drugs

In [ ]:
ukb_prescriptions = pd.read_csv('data/raw/drug_export/data/gp_scripts.txt', sep='\t')
presner = pd.read_json("data/raw/drug_export/out/result.json")

The code below is tailored to address a specific challenge in pharmacological data processing: the elimination of redundant compound entries from a dataset of drugs. This issue arises when drugs, which are often composed of various compounds, are decomposed into their constituent chemical entities. In this decomposition process, it's not uncommon to find that multiple compounds associated with a single drug actually share the same parent active compound. 

Such duplication can occur due to the presence of different salts, esters, or isomers of the same active molecule, which are chemically distinct but therapeutically equivalent. For instance, a drug might be listed with both its hydrochloride and sulfate salts, but both these forms have the same active parent compound.

The code addresses this issue by first extracting the unique compound identifiers (ChEMBL IDs) from a comprehensive dataset of drugs. It then employs a batch processing approach to query an external database (ChEMBL), retrieving detailed information about these compounds, including their hierarchical relationships (i.e., which compounds are parent compounds and which are derivatives)

In [ ]:
def fetch_chembl_data(chembl_ids_batch):
    """
    Fetches molecule forms data from the ChEMBL database for a batch of ChEMBL IDs.

    Args:
    chembl_ids_batch (list): A list of ChEMBL IDs.

    Returns:
    list: A list of molecule forms data for the given ChEMBL IDs.
    """
    url = f'https://www.ebi.ac.uk/chembl/api/data/molecule_form/set/{";".join(chembl_ids_batch)}?format=json'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['molecule_forms']
    else:
        print("Error:", response.status_code)
        return []

@lru_cache(maxsize=1024)
def get_unique_chembl_ids(chembl_ids):
    """
    Filters and returns unique ChEMBL IDs, excluding those whose parent IDs are also in the list.

    Args:
    chembl_ids (list): A list of ChEMBL IDs.

    Returns:
    list: A list of unique ChEMBL IDs after filtering.
    """
    unique_ids = set(chembl_ids)
    return [uid for uid in unique_ids if lut_parent_id.get(uid) not in unique_ids]

# Process the data to extract unique ChEMBL IDs and group them
unique_chembl_ids = presner['chemblid'].unique().tolist()
grouped_lists = presner.groupby('fk')['chemblid'].apply(tuple).reset_index(name='chemblid_list')

# Define the batch size for API requests
batch_size = 50

# Split the unique ChEMBL IDs into manageable batches
chembl_id_batches = [unique_chembl_ids[i:i + batch_size] for i in range(0, len(unique_chembl_ids), batch_size)]

# Initialize a Look-Up Table (LUT) for parent IDs
lut_parent_id = {}

# Fetch data for each batch and update the LUT
for batch in tqdm(chembl_id_batches, desc="Fetching ChEMBL Data"):
    batch_data = fetch_chembl_data(batch)
    for molecule in batch_data:
        if not molecule['is_parent']:
            lut_parent_id[molecule['molecule_chembl_id']] = molecule['parent_chembl_id']

# Create a dictionary to determine which IDs to keep
to_keep = {
    fk: get_unique_chembl_ids(tuple(sorted(chemblid_list)))
    for fk, chemblid_list in zip(grouped_lists['fk'], grouped_lists['chemblid_list'])
}

# Map the chemblid_list to each fk in the original df
presner['to_keep'] = presner['fk'].map(to_keep)

# Check if each chemblid is in the corresponding to_keep list
presner['keep_row'] = presner.apply(lambda x: x['chemblid'] in x['to_keep'], axis=1)

# Filter the dataframe to keep only the rows that are marked to keep
presner = presner[presner['keep_row']].drop(columns=['to_keep', 'keep_row'])

In [ ]:
presner = presner.set_index(['TEXT_drug_name', 'TEXT_quantity'])
ukb_prescriptions = ukb_prescriptions.set_index(['drug_name', 'quantity'])
ukb_prescriptions.index.names = ['TEXT_drug_name', 'TEXT_quantity']

ukb_prescriptions = ukb_prescriptions.join(presner, how='inner', lsuffix='_ukb', rsuffix='_presner')
ukb_prescriptions = ukb_prescriptions.reset_index().set_index('eid')
ukb_prescriptions = ukb_prescriptions[['chemblid', 'issue_date']]

In [ ]:
# Remove drugs with low occurence as they could interfere with statistical learning methods
mask = ukb_prescriptions['chemblid'].map(ukb_prescriptions['chemblid'].value_counts()) >= MIN_DRUG_OCCURENCE_THRESHOLD
ukb_prescriptions = ukb_prescriptions[mask]

In [ ]:
ukb_prescriptions.columns = ['event', 'date']
ukb_prescriptions = handle_event_df(ukb_prescriptions)
ukb_prescriptions.columns = ['drug_codes', 'drug_dates']

In [ ]:
exported_df = exported_df.join(ukb_prescriptions, how='left')
exported_df['drug_codes'] = exported_df['drug_codes'].progress_apply(lambda x: np.array([], dtype='<U13') if x is np.nan else x)
exported_df['drug_dates'] = exported_df['drug_dates'].progress_apply(lambda x: np.array([], dtype='<U13') if x is np.nan else x)

## Disease export

In [ ]:
disease_onset_df = pd.read_csv('data/raw/disease_onset.csv', dtype=str).set_index('eid')
disease_onset_df.index = disease_onset_df.index.astype(int)

In [ ]:
disease_onset_df = disease_onset_df.melt(var_name='event', value_name='date', ignore_index=False)

In [ ]:
disease_onset_df = handle_event_df(disease_onset_df)
disease_onset_df.columns = ['disease_codes', 'disease_dates']

In [ ]:
exported_df = exported_df.join(disease_onset_df, how='left')
exported_df['disease_codes'] = exported_df['disease_codes'].progress_apply(lambda x: np.array([], dtype='<U13') if x is np.nan else x)
exported_df['disease_dates'] = exported_df['disease_dates'].progress_apply(lambda x: np.array([], dtype='<U13') if x is np.nan else x)

# Save df as parquet

In [ ]:
exported_df.to_parquet('data/processed/ukb.parquet')

# Save with datasets

In [ ]:
from datasets import Features, Value, Sequence, ClassLabel, Dataset

def get_dataset_dtype(dtype):
    if pd.api.types.is_float_dtype(dtype):
        return Value('float32')
    if pd.api.types.is_integer_dtype(dtype):
        return Value('int32')
    if pd.api.types.is_string_dtype(dtype):
        return Value('string')
    if pd.api.types.is_categorical_dtype(dtype):
        return ClassLabel(names=dtype.categories.tolist())

    raise TypeError

features = {c:get_dataset_dtype(exported_df[c].dtype) for c in exported_df}
features['eid'] = Value('int32')
features['disease_codes'] =  Sequence(Value('string'))
features['disease_dates'] =  Sequence(Value('string'))
features['drug_codes'] =  Sequence(Value('string'))
features['drug_dates'] =  Sequence(Value('string'))

for c in exported_df:
    if pd.api.types.is_categorical_dtype(exported_df[c].dtype):
        exported_df[c] = exported_df[c].astype('object')

dataset = Dataset.from_pandas(exported_df, features=Features(features))
dataset.save_to_disk("data/processed/dataset")

# Additional helper files

## Special values for tokenization

In [ ]:
with open('data/processed/helper_files/special_field_values.json', 'w') as f:
    json.dump(special_values, f, indent=4)

## Field to modality dict for constructing different token sequenses

In [ ]:
# From personal:
# remove 34: Year of birth and 52: Month of birth, and add 33: Date of birth
# remove 21003: Age at recruitment, only needed for event dates, not as model input
# remove 21000: Ethnic background, 6138: Qualicifactions, they are only used for filtering not as model input
# replace 189 with 22189, the old downloaded UKB data contains this field under this id

field_to_modality = {}

for f in fields_by_file:
    for field_id in fields_by_file[f]:
        field_to_modality[field_id] = 'lab' if 'lab' in f else 'personal'

del field_to_modality[21003]
del field_to_modality[34]
del field_to_modality[52]
del field_to_modality[21000]
del field_to_modality[6138]
field_to_modality[33] = 'personal'
field_to_modality[22189] = field_to_modality[189]
del field_to_modality[189]

with open('data/processed/helper_files/field_to_modality.json', 'w') as f:
    json.dump(field_to_modality, f, indent=4)

## Field to time-invariancy for constructing date sequences during tokenization

In [ ]:
field_to_timevariant = {}

for f in fields_by_file:
    for field_id in fields_by_file[f]:
        field_to_timevariant[field_id] = False if 'invariant' in f else True

# Similarly to above
del field_to_timevariant[21003]
del field_to_timevariant[34]
del field_to_timevariant[52]
del field_to_timevariant[21000]
del field_to_timevariant[6138]
field_to_timevariant[33] = False
field_to_timevariant[22189] = field_to_timevariant[189]
del field_to_timevariant[189]


with open('data/processed/helper_files/field_timevariant.json', 'w') as f:
    json.dump(field_to_timevariant, f, indent=4)

## Convert Hierarchical String Encoding 19 (ICD10) codes, into conversion dictionaries for tokenizer

In [ ]:
ehierstr_df = pd.read_csv('data/raw/ehierstring.txt', sep='\t')
icd_hier_df = ehierstr_df[ehierstr_df['encoding_id'] == 19]

# Create a helper function to build the hierarchy recursively
def build_hierarchy(df, node_id, depth):
    node = df[df['code_id'] == node_id].iloc[0]

    children = df[df['parent_id'] == node_id] if depth != 0 else pd.Series([], dtype=int)
        
    node_dict = {
        'code_id': node['code_id'],
        'value': node['value'],
        'meaning': node['meaning'],
        'children': [build_hierarchy(df, child['code_id'], depth - 1) for _, child in children.iterrows()] if not children.empty else []
    }
    return node_dict

# Find root nodes (parent_id = 0)
root_nodes = icd_hier_df[icd_hier_df['parent_id'] == 0]

# Initialize a dictionary to store the hierarchical structure
hierarchical_structure  = []

# Create the final list of dictionaries
for _, root_node in root_nodes.iterrows():
    hierarchical_structure.append(build_hierarchy(icd_hier_df, root_node['code_id'], depth=2))


# Initialize dictionaries
code_to_chapter = {}
code_to_block = {}
block_to_chapter = {}
chapter_to_name = {}
block_to_name = {}
code_to_name = {}

# Iterate through the hierarchical_structure
for chapter in hierarchical_structure:
    chapter_value = chapter['value'][8:]  # remove "Chapter " from the front
    chapter_meaning = chapter['meaning']

    # Populate chapter_to_name dictionary
    chapter_to_name[chapter_value] = chapter_meaning

    # Iterate through blocks within the chapter
    for block in chapter.get('children', []):
        block_value = block['value'][6:] # remove "Block " from the front
        block_meaning = block['meaning']

        # Populate block_to_chapter dictionary
        block_to_chapter[block_value] = chapter_value
        block_to_name[block_value] = block_meaning

        # Iterate through codes within the block
        for code in block.get('children', []):
            code_value = code['value']
            code_meaning = code['meaning']

            # Populate code_to_chapter and code_to_block dictionaries
            code_to_chapter[code_value] = chapter_value
            code_to_block[code_value] = block_value

            # Populate code_to_name dictionary
            code_to_name[code_value] = code_meaning

with open('data/processed/helper_files/icd10_hierarchy.json', 'w') as f:
    json.dump({
        'code_to_chapter': code_to_chapter,
        'code_to_block': code_to_block,
        'block_to_chapter': block_to_chapter,
        'chapter_to_name': chapter_to_name,
        'block_to_name': block_to_name,
        'code_to_name': code_to_name,
    }, f, indent=4)